<a href="https://colab.research.google.com/github/HarshESC/TalkData-Mobile-Demographics/blob/main/Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest, f_regression, chi2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import neighbors
from sklearn import svm
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn import ensemble

import warnings
warnings.filterwarnings('ignore')
from google.colab import drive 
drive.mount('/content/gdrive')
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/"
%cd "/content/gdrive/My Drive/Final Capstone"


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Final Capstone


# Small EDA

In [2]:
master_df = pd.read_csv("master_clean.csv")

master_df.head()

,event_id,app_id,is_active,device_id,label_id_x,download_id,251,252,253,254,256,262,263,302,303,306,326,405,406,407,548,549,551,558,562,564,565,566,english_phone_brand_asus,english_phone_brand_coolpad,english_phone_brand_hammer,english_phone_brand_huawei,english_phone_brand_lshi,english_phone_brand_meizu,english_phone_brand_oppo,english_phone_brand_qiku,english_phone_brand_samsung,english_phone_brand_vivo,english_phone_brand_xiaomi,device_model_Galaxy Note 3,device_model_Galaxy Note 4,device_model_Galaxy S5,device_model_Galaxy S6,device_model_MI 3,device_model_MI 4,device_model_MX4,device_model_MX4 Pro,device_model_MX5,device_model_Mate 7,device_model_P8 Lite,device_model_P8max,device_model_R7 Plus,device_model_U3,device_model_X5SL,device_model_ZenFone 2,device_model_note顶配版,device_model_坚果手机,device_model_大神Note3,device_model_小米4C,device_model_小米note,device_model_红米2,device_model_红米2A,device_model_红米Note2,device_model_红米Note3,device_model_红米note,device_model_荣耀3C,device_model_荣耀6,device_model_荣耀6 Plus,device_model_荣耀7,device_model_荣耀7i,device_model_荣耀X2,device_model_荣耀畅玩4C,device_model_荣耀畅玩5X,device_model_超级手机1,device_model_超级手机1 Pro,device_model_超级手机1s,device_model_青春版,device_model_魅蓝NOTE,device_model_魅蓝metal,num_group
0,198,-8807740666788515175,0,6984152891252102027,548,-1823587775536413148,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
1,198,-7415261454408512527,0,6984152891252102027,254,-431108563156410500,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
2,198,-6538188762242952077,1,6984152891252102027,251,445964129009149950,1,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
3,198,-6493194103110420302,0,6984152891252102027,253,490958788141681725,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
4,198,-6349602347932703147,0,6984152891252102027,405,634550543319398880,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4


In [3]:
# Dropping AppID as I don't think it will be helpful for our question
master_df = master_df.drop(columns=["app_id"])


In [4]:

master_df["num_group"].value_counts()


5     2842
4     2690
3     1366
1     1250
2      885
11     857
10     683
0      637
9      403
8      282
6      194
7      142
Name: num_group, dtype: int64

In [5]:
numerical_columns = ["device_id", "event_id"]


# Select K Best

In [6]:
# splitting up the data and choosing num_group as target variable
X = master_df.drop(columns=["num_group"])
y = master_df["num_group"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=13, stratify=y
)


In [7]:
# set up preprocessing for pipeline (used to scale features)
preprocessing = ColumnTransformer(
    [("scale", StandardScaler(), numerical_columns),], remainder="passthrough",
)



## Random Forest Classifier

In [8]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("rf", RandomForestClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing rf, total=   1.6s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('rf',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

Grid search

In [9]:
# Max depth of 50 provides a better score, this is much better than the previous score of around 0.2.
# To improve score suggest increasing max depth even more
grid = {
    "rf__max_depth": [50, 70, 90, 110],
    "rf__n_estimators": [1, 10, 100],
    "rf__min_samples_leaf": [1, 3, 5, 7],
    "rf__criterion": ["gini"],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, verbose=True)
model.fit(X_train, y_train)


Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed:  1.3min finished


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ................ (step 2 of 2) Processing rf, total=   1.5s


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['device_id',
                                                                 

In [10]:
model.best_params_


{'rf__criterion': 'gini',
 'rf__max_depth': 90,
 'rf__min_samples_leaf': 1,
 'rf__n_estimators': 100}

In [11]:
rfc1 = ensemble.RandomForestClassifier()
params = {'rf__criterion': 'gini',
 'rf__max_depth': 70,
 'rf__min_samples_leaf': 1,
 'rf__n_estimators': 100}


X = master_df.drop(columns=["num_group"])
y = master_df["num_group"]

rfc1.fit(X_train,y_train)
predict_test_rfc = rfc1.predict(X_test)

score = cross_val_score(rfc1, X, y, cv=5)
print("Cross validation Score : ",score)
print("Variance : ",format(score.std()**2, 'f'))

Cross validation Score :  [0.29546383 0.42600164 0.29476697 0.29681112 0.2007359 ]
Variance :  0.005150


In [12]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)




,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,126,0,1,0,0,0,0,0,0,0,0,0
Actual M23-26,0,247,0,1,1,0,0,0,0,1,0,0
Actual M27-28,0,0,176,0,1,0,0,0,0,0,0,0
Actual M29-31,0,2,0,263,3,3,0,1,0,0,1,0
Actual M32-38,0,0,0,0,535,2,0,1,0,0,0,0
Actual M39+,0,0,0,0,0,568,0,0,0,0,1,0
Actual F23-,0,0,0,2,0,0,37,0,0,0,0,0
Actual F24-26,0,0,0,0,2,0,0,25,0,0,1,0
Actual F27-28,0,1,0,0,0,2,0,0,53,0,0,0
Actual F29-32,0,0,0,0,0,0,0,0,0,81,0,0


In [13]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       127
           1       0.99      0.99      0.99       250
           2       0.99      0.99      0.99       177
           3       0.98      0.96      0.97       273
           4       0.98      0.99      0.99       538
           5       0.98      1.00      0.99       569
           6       1.00      0.95      0.97        39
           7       0.93      0.89      0.91        28
           8       1.00      0.95      0.97        56
           9       0.99      1.00      0.99        81
          10       0.98      0.99      0.98       137
          11       1.00      0.95      0.98       172

    accuracy                           0.98      2447
   macro avg       0.98      0.97      0.98      2447
weighted avg       0.98      0.98      0.98      2447



#K-Nearest Neighbor Classification

In [14]:
pipeline = Pipeline(
    [("preprocessing", preprocessing), ("knn", KNeighborsClassifier())], verbose=True
)

pipeline.fit(X_train, y_train)


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.1s


Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('scale',
                                                  StandardScaler(copy=True,
                                                                 with_mean=True,
                                                                 with_std=True),
                                                  ['device_id', 'event_id'])],
                                   verbose=False)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=True)

In [15]:
grid = {
    "knn__n_neighbors": [50, 100, 150],
    "knn__weights": ["distance"],
    "knn__leaf_size": [1, 10, 20],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, verbose=2, cv=2)
model.fit(X_train, y_train)


Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:   12.0s finished


[Pipeline] ..... (step 1 of 2) Processing preprocessing, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.2s


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessing',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('scale',
                                                                         StandardScaler(copy=True,
                                                                                        with_mean=True,
                                                                                        with_std=True),
                                                                         ['device_id',
                                                                    

In [16]:
# It seems like the higher the amount of n_neighbors the better the score, for next test, will increase n_neighbors
model.best_params_


{'knn__leaf_size': 1, 'knn__n_neighbors': 150, 'knn__weights': 'distance'}

In [17]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,18,8,2,6,34,46,3,0,0,1,4,5
Actual M23-26,8,38,11,21,68,78,1,1,5,4,8,7
Actual M27-28,2,11,25,8,54,56,2,0,2,1,10,6
Actual M29-31,10,23,18,33,83,76,2,0,4,5,14,5
Actual M32-38,15,34,26,45,180,175,4,2,6,6,18,27
Actual M39+,24,31,22,40,167,221,4,4,3,12,16,25
Actual F23-,0,4,2,3,12,11,5,1,0,1,0,0
Actual F24-26,1,2,0,4,9,5,0,3,0,0,0,4
Actual F27-28,3,5,1,5,12,19,0,1,3,1,4,2
Actual F29-32,4,6,4,3,20,34,1,0,2,1,4,2


In [18]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))


Classification Report:

              precision    recall  f1-score   support

           0       0.19      0.14      0.16       127
           1       0.21      0.15      0.18       250
           2       0.20      0.14      0.17       177
           3       0.17      0.12      0.14       273
           4       0.25      0.33      0.29       538
           5       0.26      0.39      0.31       569
           6       0.22      0.13      0.16        39
           7       0.23      0.11      0.15        28
           8       0.11      0.05      0.07        56
           9       0.03      0.01      0.02        81
          10       0.10      0.07      0.08       137
          11       0.11      0.06      0.08       172

    accuracy                           0.22      2447
   macro avg       0.17      0.14      0.15      2447
weighted avg       0.20      0.22      0.21      2447



# PCA Models

In [19]:
# Since I am performing pca its necessary to standardize data and fit it to data frame.
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)

# Computing the principal components
pca = PCA()
X_pca = pca.fit_transform(X_train_std)


In [20]:
# I plan to use the amount of components that add up to 90% variance.
n_components = np.sum(np.cumsum(pca.explained_variance_ratio_) < 0.90)
X_pca = X_pca[:, :n_components]


In [21]:
# check to see the amount of components being used
n_components


47

# Random Forest Classifier

In [22]:
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=n_components)),
        ("rf", RandomForestClassifier()),
    ]
)
pipeline = pipeline.fit(X_train, y_train)


GridSearch

In [23]:
grid = {
    "rf__max_depth": [90, 110, 130],
    "rf__n_estimators": [1, 10, 100],
    "rf__min_samples_leaf": [1, 3, 5, 7],
    "rf__criterion": ["gini"],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, cv=2, verbose=2)
model.fit(X_train, y_train)


Fitting 2 folds for each of 36 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:   43.3s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:  1.2min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=47, random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('rf',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                      

In [24]:
model.best_params_


{'rf__criterion': 'gini',
 'rf__max_depth': 90,
 'rf__min_samples_leaf': 1,
 'rf__n_estimators': 100}

In [25]:
rfc1 = ensemble.RandomForestClassifier()
params = {'rf__criterion': 'gini',
 'rf__max_depth': 130,
 'rf__min_samples_leaf': 1,
 'rf__n_estimators': 100}


X = master_df.drop(columns=["num_group"])
y = master_df["num_group"]

rfc1.fit(X_train,y_train)
predict_test_rfc = rfc1.predict(X_test)

score = cross_val_score(rfc1, X, y, cv=5)
print("Cross validation Score : ",score)
print("Variance : ",format(score.std()**2, 'f'))

Cross validation Score :  [0.30363711 0.44562551 0.28250204 0.30744072 0.19664759]
Variance :  0.006401


In [26]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actual M22-",
        "Actual M23-26",
        "Actual M27-28",
        "Actual M29-31",
        "Actual M32-38",
        "Actual M39+",
        "Actual F23-",
        "Actual F24-26",
        "Actual F27-28",
        "Actual F29-32",
        "Actual F33-42",
        "Actual F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actual M22-,83,5,2,7,13,17,0,0,0,0,0,0
Actual M23-26,5,194,6,8,9,17,1,0,0,2,4,4
Actual M27-28,4,16,140,1,5,8,0,0,0,0,0,3
Actual M29-31,8,6,6,180,32,27,0,1,3,0,6,4
Actual M32-38,7,3,2,23,419,50,1,5,0,6,6,16
Actual M39+,11,11,1,22,52,435,1,3,3,9,20,1
Actual F23-,0,0,1,2,5,8,21,0,0,0,2,0
Actual F24-26,1,1,0,0,5,3,0,14,0,0,4,0
Actual F27-28,0,4,0,7,4,7,0,0,34,0,0,0
Actual F29-32,3,0,1,1,7,11,0,0,0,57,1,0


In [27]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.66      0.65      0.66       127
           1       0.79      0.78      0.78       250
           2       0.86      0.79      0.83       177
           3       0.67      0.66      0.66       273
           4       0.71      0.78      0.74       538
           5       0.71      0.76      0.73       569
           6       0.84      0.54      0.66        39
           7       0.61      0.50      0.55        28
           8       0.83      0.61      0.70        56
           9       0.76      0.70      0.73        81
          10       0.69      0.72      0.70       137
          11       0.75      0.57      0.65       172

    accuracy                           0.72      2447
   macro avg       0.74      0.67      0.70      2447
weighted avg       0.73      0.72      0.72      2447



#K Nearest Neighbor Classifier

In [28]:
pipeline = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=n_components)),
        ("knn", KNeighborsClassifier()),
    ]
)
pipeline = pipeline.fit(X_train, y_train)


Grid Search

In [29]:
grid = {
    "knn__n_neighbors": [50, 100, 150],
    "knn__weights": ["distance"],
    "knn__leaf_size": [1, 10, 20],
}

model = GridSearchCV(pipeline, grid, n_jobs=-1, cv=2, verbose=2)
model.fit(X_train, y_train)


Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.6min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=47, random_state=None,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)),
                                       ('knn',
                                        KNeighborsClassifier(algorithm='auto',
                                                             leaf_size=30,
                                                             metric='minkowski',
                                         

In [30]:
model.best_params_


{'knn__leaf_size': 20, 'knn__n_neighbors': 50, 'knn__weights': 'distance'}

In [31]:
y_pred = model.predict(X_test)

con_mat = pd.DataFrame(
    confusion_matrix(y_test, y_pred),
    index=[
        "Actually M22-",
        "Actually M23-26",
        "Actually M27-28",
        "Actually M29-31",
        "Actually M32-38",
        "Actually M39+",
        "Actually F23-",
        "Actually F24-26",
        "Actually F27-28",
        "Actually F29-32",
        "Actually F33-42",
        "Actually F43+",
    ],
    columns=[
        "Predicted M22-",
        "Predicted M23-26",
        "Predicted M27-28",
        "Predicted M29-31",
        "Predicted M32-38",
        "Predicted M39+",
        "Predicted F23-",
        "Predicted F24-26",
        "Predicted F27-28",
        "Predicted F29-32",
        "Predicted F33-42",
        "Predicted F43+",
    ],
)
con_mat.style.background_gradient(axis=None)


,Predicted M22-,Predicted M23-26,Predicted M27-28,Predicted M29-31,Predicted M32-38,Predicted M39+,Predicted F23-,Predicted F24-26,Predicted F27-28,Predicted F29-32,Predicted F33-42,Predicted F43+
Actually M22-,62,7,1,6,21,25,0,3,0,2,0,0
Actually M23-26,7,156,4,2,32,37,0,0,0,4,7,1
Actually M27-28,5,26,88,7,16,20,0,0,0,0,7,8
Actually M29-31,15,17,4,108,54,61,1,0,3,0,4,6
Actually M32-38,4,4,3,28,379,78,0,4,1,9,4,24
Actually M39+,20,23,0,19,81,397,1,1,0,9,14,4
Actually F23-,0,1,0,2,6,9,20,0,0,0,1,0
Actually F24-26,2,0,0,3,2,10,0,10,0,0,1,0
Actually F27-28,0,10,0,8,8,7,0,0,19,0,1,3
Actually F29-32,1,1,5,4,10,23,0,0,0,35,2,0


In [32]:
print("Classification Report:")
print()
print(classification_report(y_test, y_pred))

Classification Report:

              precision    recall  f1-score   support

           0       0.52      0.49      0.50       127
           1       0.63      0.62      0.63       250
           2       0.79      0.50      0.61       177
           3       0.53      0.40      0.45       273
           4       0.56      0.70      0.63       538
           5       0.53      0.70      0.60       569
           6       0.71      0.51      0.60        39
           7       0.56      0.36      0.43        28
           8       0.79      0.34      0.47        56
           9       0.58      0.43      0.50        81
          10       0.56      0.40      0.47       137
          11       0.58      0.39      0.47       172

    accuracy                           0.57      2447
   macro avg       0.61      0.49      0.53      2447
weighted avg       0.58      0.57      0.56      2447

